Example 4.2 of [MWL20].

[WML19] Wang, Jie, Victor Magron, and Jean-Bernard Lasserre. "TSSOS: A Moment-SOS hierarchy that exploits term sparsity." arXiv preprint arXiv:1912.08899 (2019).

In [5]:
using DynamicPolynomials
@polyvar x[1:3]
f = 1 + x[1]^4 + x[2]^4 + x[3]^4 + prod(x) + x[2]

x₁⁴ + x₂⁴ + x₃⁴ + x₁x₂x₃ + x₂ + 1

In [7]:
using SumOfSquares
SumOfSquares.Certificate.monomials_half_newton_polytope(monomials(f), tuple())

10-element MonomialVector{true}:
 x₁²
 x₁x₂
 x₁x₃
 x₂²
 x₂x₃
 x₃²
 x₁
 x₂
 x₃
 1

In [10]:
monos = Set(monomials(x, 0:2))

Set{Monomial{true}} with 10 elements:
  x₁²
  x₂²
  x₁x₃
  x₂x₃
  x₃²
  x₁
  x₃
  x₁x₂
  x₂
  1

In [14]:
P = Set(monomials(f))
for mono in monos
    push!(P, mono^2)
end
P

Set{Monomial{true}} with 12 elements:
  x₃⁴
  x₂²x₃²
  x₁²
  x₁²x₃²
  x₂⁴
  x₂
  x₁²x₂²
  x₂²
  x₁x₂x₃
  x₃²
  x₁⁴
  1

In [17]:
#const CEG = SumOfSquares.Certificate.ChordalExtensionGraph
g = CEG.LabelledGraph{monomialtype(f)}()
for mono in monos
    CEG.add_node!(g, mono)
end
for a in monos
    for b in monos
        if a != b && (a * b) in P
            CEG.add_edge!(g, a, b)
        end
    end
end
g

Monomial{true}-LabelledGraph with
Nodes:
x[1]^2, x[2]^2, x[1]*x[3], x[2]*x[3], x[3]^2, x[1], x[3], x[1]*x[2], x[2], 1, 
Edges:
( x[1]^2, 1 )
( x[1]^2, x[2]^2 )
( x[1]^2, x[3]^2 )
( x[2]^2, 1 )
( x[2]^2, x[3]^2 )
( x[2]^2, x[1]^2 )
( x[1]*x[3], x[2] )
( x[2]*x[3], x[1] )
( x[3]^2, 1 )
( x[3]^2, x[2]^2 )
( x[3]^2, x[1]^2 )
( x[1], x[2]*x[3] )
( x[3], x[1]*x[2] )
( x[1]*x[2], x[3] )
( x[2], 1 )
( x[2], x[1]*x[3] )
( 1, x[2] )
( 1, x[2]^2 )
( 1, x[3]^2 )
( 1, x[1]^2 )


In [18]:
H, cliques = CEG.chordal_extension(g, CEG.GreedyFillIn())

(Monomial{true}-LabelledGraph with
Nodes:
x[1]^2, x[2]^2, x[1]*x[3], x[2]*x[3], x[3]^2, x[1], x[3], x[1]*x[2], x[2], 1, 
Edges:
( x[1]^2, 1 )
( x[1]^2, x[2]^2 )
( x[1]^2, x[3]^2 )
( x[2]^2, 1 )
( x[2]^2, x[3]^2 )
( x[2]^2, x[1]^2 )
( x[1]*x[3], x[2] )
( x[2]*x[3], x[1] )
( x[3]^2, 1 )
( x[3]^2, x[2]^2 )
( x[3]^2, x[1]^2 )
( x[1], x[2]*x[3] )
( x[3], x[1]*x[2] )
( x[1]*x[2], x[3] )
( x[2], 1 )
( x[2], x[1]*x[3] )
( 1, x[2] )
( 1, x[2]^2 )
( 1, x[3]^2 )
( 1, x[1]^2 )
, Array{Monomial{true},1}[[x₁², x₂², x₃², 1], [x₃, x₁x₂], [x₂x₃, x₁], [x₁x₃, x₂], [x₂, 1]])

In [24]:
P1 = Set{eltype(P)}()
for a in monos
    push!(P1, a^2)
    for bi in H.graph.neighbors[H.n2int[a]]
        b = H.int2n[bi]
        push!(P1, a * b)
    end
end
P1

Set{Monomial{true}} with 12 elements:
  x₃⁴
  x₂²x₃²
  x₁²
  x₁²x₃²
  x₂⁴
  x₂
  x₁²x₂²
  x₂²
  x₁x₂x₃
  x₃²
  x₁⁴
  1

In [23]:
P

Set{Monomial{true}} with 12 elements:
  x₃⁴
  x₂²x₃²
  x₁²
  x₁²x₃²
  x₂⁴
  x₂
  x₁²x₂²
  x₂²
  x₁x₂x₃
  x₃²
  x₁⁴
  1